In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

Invoke the LLM to generate response to the prompt.

In [ ]:
llm.invoke("What is meant by the term 'machine learning'?")

## Creating chain with LCEL (LangChain Expression Language)

LCEL is now the default way to create chains in LangChain. It has a more pipeline-like syntax and allows you to modify already-existing chains.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an English-French translator that return whatever the user says in French"),
    ("user", "{input}")
])

Chaining the prompt and language model to generate a response is simple. Here's how you can do it:

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({
    "input": "I love going to the beach"
})

In [ ]:
# add output parser to the chain to get the output in a string format

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({
    "input": "The weather is nice today"
})

In [ ]:
llm.invoke("What is new in JavaScript?")

## Creating a Retrieval Chain


## 3.1 Load the source documents

First, we will have to load the documents that will enrich our LLM prompt. We will use [this blog post](https://blog.langchain.dev/langchain-v0-1-0/) from LangChain's official website explaining the new release. OpenAI's models were not trained on this content, so the only way to ask questions about it is to build a RAG chain.

The first thing to do is to load the blog content to our vector store. We will use beautiful soup to scrap the blog post. Then we will store it in a FAISS vector store.

In [ ]:
# retrieval chain

! pip install PyPDF

In [ ]:
# importing faiss-cpu for vector database
! pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./pdf/monopoly.pdf")

docs = loader.load()

In [ ]:
docs

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
splits

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embeddings=embeddings)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

## Creating a Context-Aware LLM Chain

Here we create a chain that will answer a question given a context. For now, we are passing the context manually, but in the next step we will pass in the documents fetched from the vector store we created above

In [ ]:
# create chain for documents

from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "what is langchain 0.1.0?",
    "context": [Document(page_content="langchain 0.1.0 is the new version of a llm app development framework.")]
})

## 3.2 Create the RAG Chain

RAG stands for Retrieval-Augmented Generation. This means that we will enrich the prompt that we send to the LLM. We will use with the documents that wil will retrieve from the vector store for this. LangChain comes with the function `create_retrieval_chain` that allows you to create one of these.

In [ ]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "what is new in langchain 0.1.0"
})

In [ ]:
response['answer']

# Creating Conversational RAG Chain

Now we will create exactly the same thing as above, but we will have the AI assistant take the history of the conversation into account. In short, we will build the same chain as above but with we will take into account the previous messages in these two steps of the chain:

- When fetching the documents from the vector store. We will fetch documents related to the entire conversation and not just the latest message.
- When answering the question. We will send to the LLM the history of the conversation along the context and query.





## Creating a Conversation-Aware Retrieval Chain

This chain will return the documents related to the entire conversation and not just the latest message.

In [ ]:
# conversational retrieval chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

## Using Retrieval Chain together with Document Chain

Now we will create a document chain that contains a placeholder for the conversation history. 

This placeholder will be populated with the conversation history that we will pass as its value. We will the plug it together with the retriever chain we created above to have a conversational retrieval-augmented chain.

In [ ]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(
    retriever_chain, document_chain)

In [ ]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "What is langchain 0.1.0 about?"
})

In [ ]:
response

In [ ]:
response['answer']

Simulate conversation history

In [ ]:
chat_history = [
    HumanMessage(content="Is there anything new on Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [ ]:
response

In [ ]:
response['answer']